to cover in this notebook:
 - very short explanation of what raster data is
 - getting a demo sentinel-2 image using STAC
 - what are bands
 - making a true colour image
 - calculating indices

# working with satellite data
For this demo we will use data from the Sentinel-2 satellite. The [Sentinel-2 mission](https://sentinel.esa.int/web/sentinel/missions/sentinel-2) is a land monitoring constellation of two satellites that provide high resolution optical imagery over the majority of the Earth's terrestrial surface. The Sentinel-2 dataset is available publicly at the [AWS open data registry](https://registry.opendata.aws/sentinel-2/).

We will use pystac (a python STAC package) to find an image that was captured within a specific date range, at the location we want, and has less that XX cloud cover.

In [1]:
import pandas
import numpy
import geopandas as gpd
import rasterio as rst
from rasterio import plot
import matplotlib.image as mpimg
import pystac_client


## Finding an image via STAC

PySTAC-Client (pystac-client) is an extension of the PySTAC library. It adds support for STAC APIs and static STAC catalogs. You can use PySTAC-Client with both static catalogs and dynamic catalogs that use APIs. Its main purpose is to support STAC API endpoints like search. 
You can interact with the [Sentinel-2 Level 2A catalog here](https://radiantearth.github.io/stac-browser/#/external/earth-search.aws.element84.com/v1/collections/sentinel-2-l2a)

In [11]:
# Initialize the STAC client
url = "https://earth-search.aws.element84.com/v1"
client = pystac_client.Client.open(url)

# Define the search parameters
bbox = [145.8, -37.3, 146.0, -37.1]  # near Lake Eildon, Victoria
time_range = "2023-01-01/2023-01-31"

search = client.search(
    max_items=1,
    collections=['sentinel-2-l2a'],
    datetime=time_range,
    bbox=bbox,
    query={"eo:cloud_cover": {"lt": 20}}
)
print(f"{search.matched()} items found")
    
# Remember, only 1 item will be printed because we've set a max items limit
for item in search.items():
    print(item.id)


28 items found
S2A_55HDU_20230130_0_L2A
S2A_55HCV_20230130_0_L2A
S2A_55HDU_20230127_0_L2A
S2A_55HCV_20230127_0_L2A
S2B_55HCU_20230125_0_L2A
S2B_55HDU_20230125_0_L2A
S2B_55HCV_20230125_0_L2A
S2B_55HDV_20230125_0_L2A
S2A_55HCU_20230120_0_L2A
S2A_55HDU_20230120_0_L2A


Each item has a series of assets associated with it. Some of these assets are the COGs for the spectral bands we are interested in.

In [14]:
assets = item.assets
for k in assets.keys():
    print(k)

TypeError: 'ItemSearch' object is not iterable

Use item_collection() to convert all Items from a search into a single PySTAC ItemCollection. The ItemCollection can then be saved as a GeoJSON FeatureCollection.

In [12]:
item_collection = search.item_collection()
item_collection.save_object('data/temp/demo_stac_itemcollection.json')

## Visualise the data
(we set a max items limit in the stac search, so we will only have 1 image to work with here. But, you can run these processes iteratively over an entire collection.)
## getting data from the STAC items
now that we have the id for the satellite image, we will fetch the actual data. In this case, we are going to fetch the COGs (cloud-optimised geotiffs) for the blue, green, red and near-infrared spectral bands.

In [ ]:
bands = ['blue','green','red','nir']

### Checking bands

### True colour image

### Calculating spectral indices